# TODO
* Why is anova p-value significant for all eSTRs? That can't be right
* Why are h2 values so high?

In [1]:
# Set up
%pylab inline

# Allow us to edit fonts in Illustrator
import matplotlib
matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['pdf.use14corefonts'] = True
matplotlib.rcParams['text.usetex'] = True

# Libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats

# Params
QVAL = 0.1
ANOVATHRESH = 0.05
SHORTEN = {
    "WholeBlood": "Blood",
    "Cells-Transformedfibroblasts": "Fibr.",
    "Muscle-Skeletal": "Muscle",
    "Artery-Tibial": "Artery",
    "Adipose-Subcutaneous": "Adipose",
    "Lung": "Lung",
    "Esophagus-Mucosa": "Esophagus",
}
NSAMPLES = {
    "WholeBlood": 144,
    "Cells-Transformedfibroblasts": 128,
    "Muscle-Skeletal": 125,
    "Artery-Tibial": 108,
    "Adipose-Subcutaneous": 102,
    "Esophagus-Mucosa": 102,
    "Lung": 110
}

# Path to data
RESDIR = "/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"
TISSUES = [item for item in list(SHORTEN.keys())]

data = {}
for t in TISSUES:
    data[t] = data[t] = pd.read_csv(os.path.join(RESDIR, t, "Master.table"), sep="\t")

Populating the interactive namespace from numpy and matplotlib


In [3]:
num_genes = []
num_estrs = []
anova_pass = []
num_caviar_best = []
gcta_perc_estr = []
gcta_perc_all = []
gcta_r2_estr = []
gcta_r2_all = []
num_samples = []

for t in TISSUES:
    xall = data[t]
    xsig = xall[xall["qvalue"]<=QVAL]
    num_genes.append(xall.shape[0])
    num_estrs.append(xsig.shape[0])
    anova_pass.append(xsig[(xsig["AD.pval"]<ANOVATHRESH)].shape[0])
    num_caviar_best.append(xsig[(xsig["best.str.score"]>=xsig["top.variant.score"])].shape[0])
    gcta_perc_estr.append(np.mean(xsig[~np.isnan(xsig["cis_str_h2"])].apply(lambda x: x["cis_str_h2"]/(x["cis_str_h2"]+x["cis_snp_h2"]), 1)))
    gcta_perc_all.append(np.mean(xall[~np.isnan(xall["cis_str_h2"])].apply(lambda x: x["cis_str_h2"]/(x["cis_str_h2"]+x["cis_snp_h2"]), 1)))
    gcta_r2_estr.append(np.mean(xsig[~np.isnan(xsig["cis_str_h2"])].apply(lambda x: x["cis_str_h2"], 1)))
    gcta_r2_all.append(np.mean(xall[~np.isnan(xall["cis_str_h2"])].apply(lambda x: x["cis_str_h2"], 1)))
    num_samples.append(NSAMPLES[t])
    
mtable = pd.DataFrame({
    "00_tissue": [SHORTEN[t] for t in TISSUES],
    "0_samples": num_samples,
    "1_numgenes": num_genes,
    "2_num.estrs": num_estrs,
    "3_gcta.estr.perch2": gcta_perc_estr,
    "4_gcta.all.perch2": gcta_perc_all,
    "5_gcta.estr.r2": gcta_r2_estr,
    "6_gcta.all.r2": gcta_r2_all,
    "7_anova.pass": anova_pass,
    "8_num_caviar_best": num_caviar_best,
})

mtable

,00_tissue,0_samples,1_numgenes,2_num.estrs,3_gcta.estr.perch2,4_gcta.all.perch2,5_gcta.estr.r2,6_gcta.all.r2,7_anova.pass,8_num_caviar_best
0,Artery,108,15584,524,0.692078,0.822748,0.152765,0.051839,524,72
1,Lung,110,15711,378,0.704192,0.833933,0.151550,0.048258,378,69
2,Muscle,125,15275,304,0.721720,0.838872,0.143059,0.042749,304,38
3,Adipose,102,15918,376,0.713711,0.833176,0.172890,0.053018,376,61
4,Esophagus,102,16111,475,0.708867,0.830623,0.160987,0.053453,475,87
5,Fibr.,128,15362,955,0.664904,0.796250,0.124663,0.045223,955,151
6,Blood,144,14845,630,0.679351,0.814670,0.114938,0.038565,630,90


In [6]:
xsig.columns

Index(['gene', 'chrom', 'gene.name', 'str.id', 'best.str.start',
       'best.str.score', 'causality.score', 'top.variant', 'top.variant.score',
       'significant', 'p.wald', 'beta', 'qvalue', 'NTEST', 'beta.se',
       'AD.pval', 'cis_str_h2', 'num_snps', 'cis_str_h2_pval', 'cis_str_h2_se',
       'nsamp', 'cis_snp_h2', 'cis_snp_h2_se', 'logL'],
      dtype='object')